# Une histoire de tiroirs
> Parties de même somme d'un ensemble  

- toc:true
- branch: master
- badges: true
- author: Nathalie Weibel
- image : images/tiroirs.png
- categories: [jupyter, binaire, combinaisons]

Voici quelques extraits d'un article de Jean-Paul Delahaye, publié dans [Pour la science 483](https://www.pourlascience.fr/sd/mathematiques/trivial-mais-puissant-le-principe-des-tiroirs-9996.php) en janvier 2018 :

>Si l’on se donne 10 entiers quelconques composés de deux chiffres, il existe
>parmi eux deux sous-ensembles disjoints de
>nombres ayant la même somme. Par exemple,
>si l’on se donne {23, 35, 44, 61, 68, 70, 71, 82, 83, 95},
>on trouvera que {23, 35, 95} donne la même
>somme que {71, 82} : 23 + 35 + 95 = 153 = 71 + 82

> **Démonstration** :   
Les sous-ensembles possibles de notre ensemble de 10 entiers sont au
nombre de $2^{10}$ = 1 024, car pour constituer un tel
sous-ensemble, on opère 10  fois de suite le
choix binaire : prendre le nombre ou ne pas le
prendre.   
Quels nombres peut-on atteindre en faisant la somme des nombres de nos sous-ensembles ? Un sous-ensemble comporte de 0 à 10 nombres compris entre 10 et 99 ; la somme est donc comprise entre 0 et 10 fois 99, donc
entre 0 et 990. En numérotant 991 tiroirs de 0 à 990, et en plaçant chaque sous-ensemble possible dans le tiroir dont le numéro est la somme de ses éléments, on trouve d’après le principe des tiroirs deux sous-ensembles ayant la même somme. 

> Le problème n’est cependant pas vraiment résolu, car on nous demande de trouver des sous-ensembles disjoints ayant la même somme, et rien ne nous permet d’affirmer que les deux
sous-ensembles trouvés à l’instant le sont. Ce n’est pas très grave : si les sous-ensembles ont
des éléments en commun, on les enlève. Les deux sous-ensembles alors obtenus ont encore la même somme, car on a enlevé les mêmes éléments, et cette fois ce sont deux sous-ensembles disjoints.

Ce résultat assure l'*existence* d'au moins une solution.  
L'objectif est ici d'exhiber toutes les solutions pour un ensemble de 10 entiers aléatoires entre 10 et 99.


## Générer un tableau de 10 entiers distincts

* Les dix entiers sont compris entre 10 et 99
* Ils sont distincts

In [1]:
from random import randint
def genere_dix_entiers():
    entiers = []
    while len(entiers) < 10:
        nouvel_entier = randint(10,99)
        if nouvel_entier not in entiers :
            entiers.append(nouvel_entier)
        else:
            pass
    return(entiers)

## Générer toutes les parties d'un ensemble

Trois méthodes sont proposées : une seule suffit, mais cela permet de choisir celle qui convient le mieux au contexte.


### Méthode 1 : construction par ajout d'élements successifs

In [2]:
def partiesliste1(liste): 
    """
    genere toutes les parties de l'ensemble des éléments de liste
    
    >>> partiesliste1([1,2,3])
    [[1, 2, 3], [2, 3], [1, 3], [3], [1, 2], [2], [1], []]
    """
    parties = [[]] 
    while liste: 
        premier = liste[0] 
        liste = liste[1:] 
        construction = [x + [premier] for x in parties] + parties  
        parties = construction 
    return parties 

### Méthode 2 : construction à l'aide du binaire

In [3]:
def partiesliste2(liste):
    """
    genere toutes les parties de l'ensemble des éléments de liste
    pour une valeur du compteur i, s'il y a un '1' en j-ème position du compteur en binaire, 
    alors on ajoute le j-ème élément dans la liste correspondant à ce compteur.
    
    >>> partiesliste2([1,2,3])
    [[], [1], [2], [1, 2], [3], [1, 3], [2, 3], [1, 2, 3]]
    """
    parties = []
    for i in range(2**len(liste)):
        nouveau = []
        for j in range(len(liste)):
            if (i >> j) & 1 == 1:     # si l'écriture binaire de i comporte un 1 en j-eme position
                nouveau.append(liste[j])
        parties.append(nouveau)
    return parties

### Méthode 3 : utilisation du module itertools

In [4]:
import itertools
def partiesliste3(liste):
    """
    genere toutes les parties de l'ensemble des éléments de liste, de taille croissante
    
    >>> partiesliste3([1,2,3])
   [[], [1], [2], [3], [1, 2], [1, 3], [2, 3], [1, 2, 3]]
    """
    parties = []
    for i in range(0, len(liste)+1):
        for partie in itertools.combinations(liste, i):
            parties.append(list(partie))
    return parties

## Calculer la somme des éléments de chaque partie

In [5]:
def calcule_sommes(liste_parties):
    sommes = {}
    for partie in liste_parties:
        somme = sum(partie)
        if somme not in sommes :
            sommes[somme] = [partie]
        else :
            sommes[somme].append(partie)
    return sommes

## Rechercher les éléments d'un dictionnaire dont la valeur a une longueur d'au moins 2.

In [6]:
def recherche_sommes_communes(dico):
    sommes_communes = {}
    for cle in dico:
        if len(dico[cle]) > 1 : 
            sommes_communes[cle] = dico[cle]
    return sommes_communes

## Supprimer les listes non disjointes

In [7]:
def nettoyage(dico):
    """
    crée un dictionnaire contenant les clés de dico, 
    lorsqu'au moins deux listes disjointes ont cette valeur pour somme
    Les valeurs associées à la clé sont les paires de deux ensembles disjoints dont la somme est la clé 
    """
    sommes_nettoyees={}
    for cle in dico: 
        tableau = (dico[cle])
        for i in range(len(tableau)):
            for j in range(i+1,len(tableau)):
                if set(tableau[i]).isdisjoint(set(tableau[j])):
                    if cle not in sommes_nettoyees :
                        sommes_nettoyees[cle] = [[set(tableau[i]), set(tableau[j])]]
                    else :
                        sommes_nettoyees[cle].append([set(tableau[i]), set(tableau[j])])
                   
    return sommes_nettoyees

## Voir les solutions 

In [8]:
mes_dix_nombres = genere_dix_entiers()
mes_dix_nombres

[44, 79, 15, 74, 72, 18, 54, 19, 14, 97]

In [9]:
mes_sommes_communes = recherche_sommes_communes(calcule_sommes(partiesliste2(mes_dix_nombres)))

In [14]:
solutions = nettoyage(mes_sommes_communes)
print("Liste ordonnée des sommes que l'on peut obtenir à partir d'au moins deux sous-ensembles distincts de la liste ", mes_dix_nombres," : \n", sorted([cle for cle in solutions]))
solutions

Liste ordonnée des sommes que l'on peut obtenir à partir d'au moins deux sous-ensembles distincts de la liste  [44, 79, 15, 74, 72, 18, 54, 19, 14, 97]  : 
 [33, 72, 73, 87, 88, 91, 92, 93, 97, 98, 106, 111, 112, 116, 123, 126, 130, 133, 141, 142, 146, 148, 151, 153, 155, 159, 160, 165, 169, 170, 171, 176, 184, 185, 187, 189, 190, 191, 194, 205, 206, 207, 209, 216, 221, 234, 243]


{123: [[{44, 79}, {14, 18, 19, 72}]],
 153: [[{74, 79}, {18, 19, 44, 72}]],
 133: [[{15, 44, 74}, {54, 79}]],
 72: [[{72}, {18, 54}]],
 116: [[{44, 72}, {18, 19, 79}],
  [{44, 72}, {19, 97}],
  [{18, 44, 54}, {19, 97}]],
 151: [[{72, 79}, {15, 18, 44, 74}],
  [{72, 79}, {14, 19, 44, 74}],
  [{72, 79}, {54, 97}],
  [{15, 18, 44, 74}, {54, 97}],
  [{18, 54, 79}, {14, 19, 44, 74}],
  [{14, 19, 44, 74}, {54, 97}]],
 87: [[{15, 72}, {14, 19, 54}]],
 146: [[{72, 74}, {14, 15, 19, 44, 54}]],
 190: [[{44, 72, 74}, {14, 79, 97}], [{18, 44, 54, 74}, {14, 79, 97}]],
 205: [[{54, 72, 79}, {15, 19, 74, 97}]],
 97: [[{18, 79}, {97}]],
 141: [[{18, 44, 79}, {15, 54, 72}], [{15, 54, 72}, {44, 97}]],
 33: [[{15, 18}, {14, 19}]],
 112: [[{15, 18, 79}, {14, 44, 54}],
  [{14, 44, 54}, {15, 97}],
  [{14, 19, 79}, {15, 97}]],
 92: [[{18, 74}, {14, 15, 19, 44}]],
 171: [[{14, 15, 19, 44, 79}, {74, 97}]],
 169: [[{14, 18, 19, 44, 74}, {72, 97}]],
 184: [[{15, 18, 72, 79}, {14, 19, 54, 97}],
  [{14, 19, 72, 79

In [11]:
compteur = 0
for cle in solutions:
    compteur = compteur + len(solutions[cle])
compteur

77